In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
baseColNames  = ['Область', 'ДатаПодачиЗаявки', 'Заявитель',
                 'Поставщик', 'Продукт',
                 'Объем', 'БазоваяЕдиницаИзмерения',
                 'ПлощадьИспользованияГа', 'Статус', 'ТипЗаявки',
                 'ДатаПринятияЗаявки', 'ДатаОтклонения',
                 'СуммаСубсидийТг', 'ПричинаОтклонения',
                 'ПримечаниеКОтрицательномуРешению', 'group', 'year']
      

#---------------------------------------------------------------------------------
baseColNamesReorderForFerti = ['Область', 'Дата подачи заявки', 'Наименование заявителя',
                        'Производитель удобрений', 'Наименование удобрения', 
                        'Объем удобрения', 'Единица измерения',
                        'Площадь использования, га', 'Статус', 'Тип заявки',
                        'Дата принятия заявки', 'Дата отрицательного решения', 
                        'Сумма субсидий (расчетная), тг.', 'Причина отрицательного решения',
                        'Примечание к отрицательному решению', 'group', 'year']

#---------------------------------------------------------------------------------
baseColNamesReorderForSeeds = ['Область', 'Дата подачи заявки', 'Наименование заявителя',
                        'Поставщик семян', 'Наименование', 'Объем семян в базовой ед.измерения',
                        'Базовая единица измерения', 'Площадь использования, га', 'Статус',
                        'Тип заявки', 'Дата принятия заявки', 'Дата отклонения',
                        'Сумма субсидий (расчетная), тг.', 'Причина отклонения',
                        'Примечание к отрицательному решению', 'group']


#------------------------------------ P E S T I C I D E S ---------------------------------------
p20 =  pd.read_csv(r'\\aafssrv\SHARED-BI\files_aa_dwh_x\dataBase\qldData\apps\Пестициды2020.csv', encoding='utf-8', sep='|')
p20['group'] = "Пестициды"
p20 = p20.iloc[:, 2:]
p20['year'] = "2020"
# print(p20.columns)
print(len(p20.columns))


p19 =  pd.read_csv(r'\\aafssrv\SHARED-BI\files_aa_dwh_x\dataBase\qldData\apps\Пестициды2019.csv', encoding='utf-8', sep='|')
p19['group'] = "Пестициды"
p19 = p19.iloc[:, 2:]
p19['year'] = "2019"
# print(p19.columns)
print(len(p19.columns))


p = pd.concat([p19, p20], ignore_index = True)
p.columns = baseColNames

#------------------------------------ F E R T L I Z E R S ---------------------------------------

f20 =  pd.read_csv(r'\\aafssrv\SHARED-BI\files_aa_dwh_x\dataBase\qldData\apps\Удобрения2020.csv', encoding='utf-8', sep='|')
f20['group'] = "Удобрения"
f20 = f20.iloc[:, 2:]
f20['year'] = "2020"

f19 =  pd.read_csv(r'\\aafssrv\SHARED-BI\files_aa_dwh_x\dataBase\qldData\apps\Удобрения2019.csv', encoding='utf-8', sep='|')
f19['group'] = "Удобрения"
f19 = f19.iloc[:, 2:]
f19['year'] = "2019"

f = pd.concat([f19, f20], ignore_index = True)
f = f[baseColNamesReorderForFerti]
f.columns = baseColNames

#------------------------------------ S E E D S ------------------------------------------------
sAll =  pd.read_csv(r'\\aafssrv\SHARED-BI\files_aa_dwh_x\dataBase\qldData\apps\Семена2019_2020.csv', encoding='utf-8', sep='|')
sAll['group'] = "Семена"
sAll = sAll.iloc[:, 1:] #check columns
sAll = sAll[baseColNamesReorderForSeeds]

s19 = sAll[sAll["Дата принятия заявки"].astype(str).str.contains("2019", case = False)]
s19['year'] = "2019"
s20 = sAll[sAll["Дата принятия заявки"].astype(str).str.contains("2020", case = False)]
s20['year'] = "2020"

s = pd.concat([s19, s20], ignore_index = True)
s.columns = baseColNames
#------------------------------------ A L L -- C A T E G O R I E S ------------------------------------------------
q = pd.concat([p, f, s], ignore_index = True)
q = q[q['Область'].notna()] ########## NEED TO REPARSE TO GET AlL NFO
q = q.replace(r'\n',' ', regex=True)
q = q.replace(r'~',' ', regex=True)
q['applicantID'] =  q[['Заявитель', 'Область']].agg('-'.join, axis=1)

#------------------------------------ C O N S T R A C T I O N S ------------------------------------------------
colsToUpper = [2,3]
for colUpper in range(len(colsToUpper)):
    q.iloc[:, colsToUpper[colUpper]] = q.iloc[:, colsToUpper[colUpper]].str.upper()
q = q.replace(to_replace = ["ТОВАРИЩЕСТВА С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ", "ТОВАРИЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ",
                            'ЖАУАПКЕРШІЛІГІ ШЕКТЕУЛІ СЕРІКТЕСТІГІ'], 
              value = "ТОО",
              regex=True)
q['Заявитель'] = q['Заявитель'].replace(to_replace = ['"', "'", '«', '»', '/', '\(', '\)'], 
              value = "", regex=True)
q['Поставщик'] = q['Поставщик'].replace(to_replace = ['"', "'", '«', '»', '/', '\(', '\)'], 
              value = "", regex=True)
q['Область'] = q['Область'].replace("Шымкент г.а.",  "г.Шымкент", regex=True)
q['Область'] = q['Область'].replace("Алматы г.а.",  "г.Алматы", regex=True)
q['Область'] = q['Область'].replace("Нур-Султан г.а.",  "г.Нур-Султан", regex=True)

#------------------------------------ D I S T R I B U T O R S ------------------------------------------------
pak = ['ТОО ПОЛГАР АГРОКЕМ', 'ТОО ПЕРВАЯ АГРОХИМИЧЕСКАЯ КОМПАНИЯ']
q['Поставщик'].replace({pak[0]: 'ПАК', pak[1]: 'ПАК', 'ТОО ALEM AGRO TRADE': 'ALEM AGRO'}, inplace=True, regex=True)
q['Поставщик'] = q['Поставщик'].str.replace('ТОО ALEM AGRO HOLDING АЛЕМАГРО ХОЛДИНГ', 'ALEM AGRO')

#------------------------------------  A A   l o g i c a l  c o l u m n ------------------------------------------------- 
q['АА'] = np.where(q['Поставщик'] == 'ALEM AGRO', 'Да', 'Нет')
aaClient = q[['АА', 'applicantID']].loc[q['АА'] == "Да"].drop_duplicates().reset_index().iloc[:,2:]
q['isSubsAaClient'] = q.applicantID.isin(aaClient.applicantID).astype(bool)

#------------------------------------ P A T H  ------------------------------------
q.to_csv(r'\\aafssrv\SHARED-BI\files_aa_dwh_x\dataBase\qldData\apps\q.csv', encoding='utf-8', sep='|', quoting = csv.QUOTE_NONE, quotechar="",  escapechar="~")
#------------------------------------ R A N D O M   5   R O W S ------------------------------------
q.sample(5)

17


ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [318]:
q#.status.drop_duplicates().unique

,region,appDate,applicant,provider,product,volume,unit,areaOfUsage,status,typeOfApp,AppAcceptanceDate,DateOfRejection,subsidies,reasonOfRejection,NoteOnNegativeResolution,group,year,applicantID,АА,isSubsAaClient
0,г.Шымкент,03.12.2019 15:18,КХ BEIIBARYSFRUITS,ТОО КОСАГРОКОММЕРЦ,"ВАЛСАГЛИФ, водный раствор","60,00",литр,"9,58",Заявка оплачена,Прямая,03.12.2019 16:31,NaN,"33 530,00",NaN,NaN,Пестициды,2019,"КХ ""Beiibarysfruits""-Шымкент г.а.",Нет,False
1,Павлодарская область,03.12.2019 14:17,ТОО ПАВЛОДАРСКАЯ СЕЛЬСКОХОЗЯЙСТВЕННАЯ ОПЫТНАЯ ...,ТОО ЩЕЛКОВО АГРОХИМ-KZ,"ПРИМАДОННА, суспензионная эмульсия","350,00",литр,"1 645,55",Отрицательное решение,Прямая,NaN,23.01.2020 12:44,"0,00",несоответствие сельхозтоваропроизводителя (сел...,Некорректно заполненная заявка,Пестициды,2019,"Товарищество с ограниченной ответственностью ""...",Нет,False
2,Павлодарская область,03.12.2019 13:55,ТОО ПАВЛОДАРСКАЯ СЕЛЬСКОХОЗЯЙСТВЕННАЯ ОПЫТНАЯ ...,ТОО ЩЕЛКОВО АГРОХИМ-KZ,"ОВСЮГЕН ЭКСТРА, концентрат эмульсии","350,00",литр,"1 359,81",Отрицательное решение,Прямая,NaN,23.01.2020 12:42,"0,00",несоответствие сельхозтоваропроизводителя (сел...,Некорректно заполненная заявка,Пестициды,2019,"Товарищество с ограниченной ответственностью ""...",Нет,False
3,Павлодарская область,03.12.2019 12:46,ТОО ПАВЛОДАРСКАЯ СЕЛЬСКОХОЗЯЙСТВЕННАЯ ОПЫТНАЯ ...,ТОО ЩЕЛКОВО АГРОХИМ-KZ,"СПРУТ ЭКСТРА, водный раствор","1 250,00",литр,"978,75",Отрицательное решение,Прямая,NaN,23.01.2020 12:40,"0,00",несоответствие сельхозтоваропроизводителя (сел...,Некорректно заполненная заявка,Пестициды,2019,"Товарищество с ограниченной ответственностью ""...",Нет,False
4,Павлодарская область,03.12.2019 12:33,ТОО ПАВЛОДАРСКАЯ СЕЛЬСКОХОЗЯЙСТВЕННАЯ ОПЫТНАЯ ...,ТОО ЩЕЛКОВО АГРОХИМ-KZ,"ОВСЮГЕН ЭКСТРА, концентрат эмульсии","350,00",литр,"1 359,81",Отрицательное решение,Прямая,NaN,23.01.2020 12:39,"0,00",несоответствие сельхозтоваропроизводителя (сел...,Некорректно заполненная заявка,Пестициды,2019,"Товарищество с ограниченной ответственностью ""...",Нет,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174062,Северо-Казахстанская область,06.05.2020 16:22,ТОО АСЫЛ ЛОГИСТИКС,ООО АГРОКОМПЛЕКС КУРГАНСЕМЕНА,Боевчанка,"67 280,00",кг,"2 368,00",Заявка оплачена,Заявка на получение субсидий за фактически при...,06.05.2020 18:34,NaN,"5 247 839,00",NaN,NaN,Семена,2020,"ТОО ""АСЫЛ ЛОГИСТИКС""-Северо-Казахстанская область",Нет,False
174063,Костанайская область,06.05.2020 14:37,КХ ТРЕТЬЯК НИКОЛАЙ СЕРГЕЕВИЧ,КРЕСТЬЯНСКОЕ ХОЗЯЙСТВО ЛУГОВОЕ,ОМСКАЯ 36 ®,"20 000,20",кг,"1 210,00",Заявка оплачена,Заявка на получение субсидий за фактически при...,17.05.2020 00:48,NaN,"1 560 015,00",NaN,NaN,Семена,2020,"КХ ""Третьяк Николай Сергеевич""-Костанайская об...",Нет,False
174064,Костанайская область,06.05.2020 14:30,КРЕСТЬЯНСКОЕ ХОЗЯЙСТВО ТРЕТЬЯК СЕРГЕЙ ЕФИМОВИЧ,КРЕСТЬЯНСКОЕ ХОЗЯЙСТВО ЛУГОВОЕ,ОМСКАЯ 36 ®,"33 000,78",кг,"1 986,00",Заявка оплачена,Заявка на получение субсидий за фактически при...,17.05.2020 00:44,NaN,"2 574 060,00",NaN,NaN,Семена,2020,"Крестьянское хозяйство ""Третьяк Сергей Ефимови...",Нет,False
174065,Северо-Казахстанская область,06.05.2020 14:27,ТОО КАНТЕМИР-АГРО,ТОО АБИ-ЖЕР,СОЛАР КЛ,"2 628,00",кг,"879,00",Отозвана,Заявка на получение субсидий за фактически при...,06.05.2020 18:30,NaN,"7 358 399,00",NaN,NaN,Семена,2020,"ТОО ""КАНТЕМИР-АГРО""-Северо-Казахстанская область",Нет,False


In [16]:
server = '192.168.1.209'  #tcp:myserver.database.windows.net 
database = 'AA_DWH_X' #mydb
username = 'pbi_report' #myusername 
password = '1s22s22p6' #mypassword
conn = pyodbc.connect('DRIVER={sql server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = conn.cursor()
# conn.setdecoding(pyodbc.SQL_CHAR, encoding='utf-8')

In [17]:
sql_1 = ('SELECT * FROM information_schema.tables')   #
df_1 = pd.read_sql(sql_1,conn)
df_1

,TABLE_CATALOG,TABLE_SCHEMA,TABLE_NAME,TABLE_TYPE
0,AA_DWH_X,dbo,sysdiagrams,BASE TABLE
1,AA_DWH_X,dbo,region,BASE TABLE
2,AA_DWH_X,dbo,country,BASE TABLE
3,AA_DWH_X,dbo,taldauAreaDatas,BASE TABLE
4,AA_DWH_X,dbo,testTime,BASE TABLE
5,AA_DWH_X,dbo,admCenter,BASE TABLE
6,AA_DWH_X,dbo,TaldauDatas,BASE TABLE
7,AA_DWH_X,dbo,subsidies_02,BASE TABLE
8,AA_DWH_X,dbo,subsidies,BASE TABLE
9,AA_DWH_X,dbo,cato,BASE TABLE


In [18]:
qName_sql = ('SELECT * FROM qName')
qName = pd.read_sql(qName_sql, conn)
qName = qName.applymap(str)
qName.codERP = qName.codERP.str.pad(9, side='left', fillchar='0')
qName

,name,codERP
0,Минеральное удобрение Мегафол (Megafol) (N-3% ...,000000001
1,Минеральное удобрение Мегафол (Megafol) (N-3; ...,000000001
2,Минеральное удобрение Мегафол (Megafol) (N-3% ...,000000001
3,"ГРАНД ЭКСТРА 540, водный раствор",000000002
4,"ЭФФЕКТ, концентрат эмульсии",000000003
...,...,...
4321,ЯРЛЫК,000005550
4322,Ясон,000005554
4323,Ясон Эконом,000005554
4324,Ясон Стандарт,000005555


In [19]:
cato_sql = ('SELECT * FROM cato')
cato = pd.read_sql(cato_sql, conn)
cato = cato.applymap(str)
# cato[cato.name.str.contains("Шымкент", case = False)]
cato

,cato_Id,lvl1,lvl2,lvl3,lvl4,lvl5,kaz_name,name,nn,group
0,110000000,11,00,00,000,0,А?мола облысы,Акмолинская область,00,АКМ
1,111000000,11,10,00,000,3,К?кшетау ?.?.,Кокшетау Г.А.,01,АКМ
2,111010000,11,10,10,000,1,К?кшетау ?.,г.Кокшетау,,АКМ
3,111033000,11,10,33,000,2,Красный Яр а.о.,Красноярский с.о.,,АКМ
4,111033100,11,10,33,100,2,Красный Яр а.,с.Красный Яр,,АКМ
...,...,...,...,...,...,...,...,...,...,...
16502,790000000,79,00,00,000,1,Шымкент ?.,г.Шымкент,,ТУР
16503,791110000,79,11,10,000,1,Абай ауданы,Абайский район,,ТУР
16504,791310000,79,13,10,000,1,?л Фараби ауданы,Аль-Фарабийский район,,ТУР
16505,791510000,79,15,10,000,1,Е?бекші ауданы,Енбекшинский район,,ТУР


In [20]:
subsidies_02_raw = pd.read_csv(r"\\aafssrv\SHARED-BI\files_aa_dwh_x\dataBase\qldData\q_02.csv", encoding="utf-8", sep="|", quoting = csv.QUOTE_NONE, quotechar="",  escapechar="~")
subsidies_02_raw

,Unnamed: 0,region,appDate,applicant,provider,product,volume,unit,areaOfUsage,status,...,AppAcceptanceDate,DateOfRejection,subsidies,reasonOfRejection,NoteOnNegativeResolution,group,year,applicantID,АА,isSubsAaClient
0,0,г.Шымкент,03.12.2019 15:18,КХ BEIIBARYSFRUITS,ТОО КОСАГРОКОММЕРЦ,"ВАЛСАГЛИФ, водный раствор","60,00",литр,"9,58",Заявка оплачена,...,03.12.2019 16:31,NaN,"33 530,00",NaN,NaN,Пестициды,2019,"КХ ""Beiibarysfruits""-Шымкент г.а.",Нет,False
1,1,Павлодарская область,03.12.2019 14:17,ТОО ПАВЛОДАРСКАЯ СЕЛЬСКОХОЗЯЙСТВЕННАЯ ОПЫТНАЯ ...,ТОО ЩЕЛКОВО АГРОХИМ-KZ,"ПРИМАДОННА, суспензионная эмульсия","350,00",литр,"1 645,55",Отрицательное решение,...,NaN,23.01.2020 12:44,"0,00",несоответствие сельхозтоваропроизводителя (сел...,Некорректно заполненная заявка,Пестициды,2019,"Товарищество с ограниченной ответственностью ""...",Нет,False
2,2,Павлодарская область,03.12.2019 13:55,ТОО ПАВЛОДАРСКАЯ СЕЛЬСКОХОЗЯЙСТВЕННАЯ ОПЫТНАЯ ...,ТОО ЩЕЛКОВО АГРОХИМ-KZ,"ОВСЮГЕН ЭКСТРА, концентрат эмульсии","350,00",литр,"1 359,81",Отрицательное решение,...,NaN,23.01.2020 12:42,"0,00",несоответствие сельхозтоваропроизводителя (сел...,Некорректно заполненная заявка,Пестициды,2019,"Товарищество с ограниченной ответственностью ""...",Нет,False
3,3,Павлодарская область,03.12.2019 12:46,ТОО ПАВЛОДАРСКАЯ СЕЛЬСКОХОЗЯЙСТВЕННАЯ ОПЫТНАЯ ...,ТОО ЩЕЛКОВО АГРОХИМ-KZ,"СПРУТ ЭКСТРА, водный раствор","1 250,00",литр,"978,75",Отрицательное решение,...,NaN,23.01.2020 12:40,"0,00",несоответствие сельхозтоваропроизводителя (сел...,Некорректно заполненная заявка,Пестициды,2019,"Товарищество с ограниченной ответственностью ""...",Нет,False
4,4,Павлодарская область,03.12.2019 12:33,ТОО ПАВЛОДАРСКАЯ СЕЛЬСКОХОЗЯЙСТВЕННАЯ ОПЫТНАЯ ...,ТОО ЩЕЛКОВО АГРОХИМ-KZ,"ОВСЮГЕН ЭКСТРА, концентрат эмульсии","350,00",литр,"1 359,81",Отрицательное решение,...,NaN,23.01.2020 12:39,"0,00",несоответствие сельхозтоваропроизводителя (сел...,Некорректно заполненная заявка,Пестициды,2019,"Товарищество с ограниченной ответственностью ""...",Нет,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160742,174062,Северо-Казахстанская область,06.05.2020 16:22,ТОО АСЫЛ ЛОГИСТИКС,ООО АГРОКОМПЛЕКС КУРГАНСЕМЕНА,Боевчанка,"67 280,00",кг,"2 368,00",Заявка оплачена,...,06.05.2020 18:34,NaN,"5 247 839,00",NaN,NaN,Семена,2020,"ТОО ""АСЫЛ ЛОГИСТИКС""-Северо-Казахстанская область",Нет,False
160743,174063,Костанайская область,06.05.2020 14:37,КХ ТРЕТЬЯК НИКОЛАЙ СЕРГЕЕВИЧ,КРЕСТЬЯНСКОЕ ХОЗЯЙСТВО ЛУГОВОЕ,ОМСКАЯ 36 ®,"20 000,20",кг,"1 210,00",Заявка оплачена,...,17.05.2020 00:48,NaN,"1 560 015,00",NaN,NaN,Семена,2020,"КХ ""Третьяк Николай Сергеевич""-Костанайская об...",Нет,False
160744,174064,Костанайская область,06.05.2020 14:30,КРЕСТЬЯНСКОЕ ХОЗЯЙСТВО ТРЕТЬЯК СЕРГЕЙ ЕФИМОВИЧ,КРЕСТЬЯНСКОЕ ХОЗЯЙСТВО ЛУГОВОЕ,ОМСКАЯ 36 ®,"33 000,78",кг,"1 986,00",Заявка оплачена,...,17.05.2020 00:44,NaN,"2 574 060,00",NaN,NaN,Семена,2020,"Крестьянское хозяйство ""Третьяк Сергей Ефимови...",Нет,False
160745,174065,Северо-Казахстанская область,06.05.2020 14:27,ТОО КАНТЕМИР-АГРО,ТОО АБИ-ЖЕР,СОЛАР КЛ,"2 628,00",кг,"879,00",Отозвана,...,06.05.2020 18:30,NaN,"7 358 399,00",NaN,NaN,Семена,2020,"ТОО ""КАНТЕМИР-АГРО""-Северо-Казахстанская область",Нет,False


In [21]:
qWithProdCodes = pd.merge(subsidies_02_raw, qName[["name", "codERP"]], left_on = "product", right_on = "name", how='left').drop(columns = ["name"])
subsidies_02 = pd.merge(qWithProdCodes, cato[["name", "cato_Id"]], left_on = "region", right_on = "name", how='left').drop(columns = ["name"])

In [22]:
subsidies_02['appDate'] = subsidies_02['appDate'].apply(pd.to_datetime)#to_numeric) 

# subsidies_02[['appDate', 'AppAcceptanceDate', 'DateOfRejection']] = subsidies_02[['appDate', 'AppAcceptanceDate', 'DateOfRejection']].apply(pd.to_datetime)#to_numeric) 
subsidies_02['appDate'] = pd.to_datetime(subsidies_02['appDate'], format='%Y%m%d %H:%M:%S').dt.strftime("%Y%m%d %H:%M:%S")
# qWithKatoCodes['AppAcceptanceDate'] = pd.to_datetime(qWithKatoCodes['AppAcceptanceDate'], format='%Y%m%dT%H:%M:%S')
# qWithKatoCodes['DateOfRejection'] = pd.to_datetime(qWithKatoCodes['DateOfRejection'], format='%Y%m%dT%H:%M:%S')

subsidies_02[['applicant', 'provider', 'product', 'unit', 'status', 'typeOfApp', 'reasonOfRejection', 'NoteOnNegativeResolution', 'group', 'year', 'applicantID', 'АА', 'codERP', 'cato_Id', 'AppAcceptanceDate', 'DateOfRejection']] = subsidies_02[['applicant', 'provider', 'product', 'unit', 'status', 'typeOfApp', 'reasonOfRejection', 'NoteOnNegativeResolution', 'group', 'year', 'applicantID', 'АА', 'codERP', 'cato_Id', 'AppAcceptanceDate', 'DateOfRejection']].applymap(str)#to_numeric) 
subsidies_02[['volume', 'areaOfUsage', 'subsidies']] = subsidies_02[['volume', 'areaOfUsage', 'subsidies']].replace(to_replace = [','], value = ".", regex=True)
subsidies_02[['volume', 'areaOfUsage', 'subsidies']] = subsidies_02[['volume', 'areaOfUsage', 'subsidies']].replace(to_replace = [' '], value = "", regex=True)
# subsidies_02[['appDate', 'AppAcceptanceDate', 'DateOfRejection']] = subsidies_02[['appDate', 'AppAcceptanceDate', 'DateOfRejection']].applymap(str).replace(to_replace = ['-'], value = "", regex=True)
# subsidies_02[['appDate', 'AppAcceptanceDate', 'DateOfRejection']] = subsidies_02[['appDate', 'AppAcceptanceDate', 'DateOfRejection']].applymap(str).replace(to_replace = ['Nat'], value = "", regex=True)

subsidies_02[['volume', 'areaOfUsage', 'subsidies']] = subsidies_02[['volume', 'areaOfUsage', 'subsidies']].apply(pd.to_numeric) 
# subsidies_02 = subsidies_02.drop_duplicates()
print(subsidies_02.dtypes) 

Unnamed: 0                    int64
region                       object
appDate                      object
applicant                    object
provider                     object
product                      object
volume                      float64
unit                         object
areaOfUsage                 float64
status                       object
typeOfApp                    object
AppAcceptanceDate            object
DateOfRejection              object
subsidies                   float64
reasonOfRejection            object
NoteOnNegativeResolution     object
group                        object
year                         object
applicantID                  object
АА                           object
isSubsAaClient                 bool
codERP                       object
cato_Id                      object
dtype: object


In [36]:
subsidies_02 = subsidies_02.drop_duplicates()

In [43]:
subsidies_02.rename(columns={'Unnamed: 0': 'index'}, inplace=True)

C:\Users\M_MUKHANOVA\miniconda3\lib\site-packages\pandas\core\frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [52]:
subsidies_02['applicantIn'] = ''
subsidies_02['providerIn'] = ''
print(subsidies_02.columns)
subsidies_02

C:\Users\M_MUKHANOVA\miniconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\M_MUKHANOVA\miniconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


Index(['index', 'region', 'appDate', 'applicant', 'provider', 'product',
       'volume', 'unit', 'areaOfUsage', 'status', 'typeOfApp',
       'AppAcceptanceDate', 'DateOfRejection', 'subsidies',
       'reasonOfRejection', 'NoteOnNegativeResolution', 'group', 'year',
       'applicantID', 'АА', 'isSubsAaClient', 'codERP', 'cato_Id',
       'applicantIn', 'providerIn'],
      dtype='object')


,index,region,appDate,applicant,provider,product,volume,unit,areaOfUsage,status,...,NoteOnNegativeResolution,group,year,applicantID,АА,isSubsAaClient,codERP,cato_Id,applicantIn,providerIn
0,0,г.Шымкент,20190312 15:18:00,КХ BEIIBARYSFRUITS,ТОО КОСАГРОКОММЕРЦ,"ВАЛСАГЛИФ, водный раствор",60.00,литр,9.58,Заявка оплачена,...,nan,Пестициды,2019,"КХ ""Beiibarysfruits""-Шымкент г.а.",Нет,False,000000668,790000000,,
2,1,Павлодарская область,20190312 14:17:00,ТОО ПАВЛОДАРСКАЯ СЕЛЬСКОХОЗЯЙСТВЕННАЯ ОПЫТНАЯ ...,ТОО ЩЕЛКОВО АГРОХИМ-KZ,"ПРИМАДОННА, суспензионная эмульсия",350.00,литр,1645.55,Отрицательное решение,...,Некорректно заполненная заявка,Пестициды,2019,"Товарищество с ограниченной ответственностью ""...",Нет,False,000004567,550000000,,
4,2,Павлодарская область,20190312 13:55:00,ТОО ПАВЛОДАРСКАЯ СЕЛЬСКОХОЗЯЙСТВЕННАЯ ОПЫТНАЯ ...,ТОО ЩЕЛКОВО АГРОХИМ-KZ,"ОВСЮГЕН ЭКСТРА, концентрат эмульсии",350.00,литр,1359.81,Отрицательное решение,...,Некорректно заполненная заявка,Пестициды,2019,"Товарищество с ограниченной ответственностью ""...",Нет,False,000001116,550000000,,
6,3,Павлодарская область,20190312 12:46:00,ТОО ПАВЛОДАРСКАЯ СЕЛЬСКОХОЗЯЙСТВЕННАЯ ОПЫТНАЯ ...,ТОО ЩЕЛКОВО АГРОХИМ-KZ,"СПРУТ ЭКСТРА, водный раствор",1250.00,литр,978.75,Отрицательное решение,...,Некорректно заполненная заявка,Пестициды,2019,"Товарищество с ограниченной ответственностью ""...",Нет,False,000001335,550000000,,
8,4,Павлодарская область,20190312 12:33:00,ТОО ПАВЛОДАРСКАЯ СЕЛЬСКОХОЗЯЙСТВЕННАЯ ОПЫТНАЯ ...,ТОО ЩЕЛКОВО АГРОХИМ-KZ,"ОВСЮГЕН ЭКСТРА, концентрат эмульсии",350.00,литр,1359.81,Отрицательное решение,...,Некорректно заполненная заявка,Пестициды,2019,"Товарищество с ограниченной ответственностью ""...",Нет,False,000001116,550000000,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
312193,174062,Северо-Казахстанская область,20200605 16:22:00,ТОО АСЫЛ ЛОГИСТИКС,ООО АГРОКОМПЛЕКС КУРГАНСЕМЕНА,Боевчанка,67280.00,кг,2368.00,Заявка оплачена,...,nan,Семена,2020,"ТОО ""АСЫЛ ЛОГИСТИКС""-Северо-Казахстанская область",Нет,False,000002909,590000000,,
312195,174063,Костанайская область,20200605 14:37:00,КХ ТРЕТЬЯК НИКОЛАЙ СЕРГЕЕВИЧ,КРЕСТЬЯНСКОЕ ХОЗЯЙСТВО ЛУГОВОЕ,ОМСКАЯ 36 ®,20000.20,кг,1210.00,Заявка оплачена,...,nan,Семена,2020,"КХ ""Третьяк Николай Сергеевич""-Костанайская об...",Нет,False,000004383,390000000,,
312197,174064,Костанайская область,20200605 14:30:00,КРЕСТЬЯНСКОЕ ХОЗЯЙСТВО ТРЕТЬЯК СЕРГЕЙ ЕФИМОВИЧ,КРЕСТЬЯНСКОЕ ХОЗЯЙСТВО ЛУГОВОЕ,ОМСКАЯ 36 ®,33000.78,кг,1986.00,Заявка оплачена,...,nan,Семена,2020,"Крестьянское хозяйство ""Третьяк Сергей Ефимови...",Нет,False,000004383,390000000,,
312199,174065,Северо-Казахстанская область,20200605 14:27:00,ТОО КАНТЕМИР-АГРО,ТОО АБИ-ЖЕР,СОЛАР КЛ,2628.00,кг,879.00,Отозвана,...,nan,Семена,2020,"ТОО ""КАНТЕМИР-АГРО""-Северо-Казахстанская область",Нет,False,000004957,590000000,,


In [53]:
qq = subsidies_02.loc[subsidies_02['applicant'].str.contains('Ғ')]
qqqq = qq.sample(3)
qqqq

,index,region,appDate,applicant,provider,product,volume,unit,areaOfUsage,status,...,NoteOnNegativeResolution,group,year,applicantID,АА,isSubsAaClient,codERP,cato_Id,applicantIn,providerIn
32945,18497,Кызылординская область,20190813 17:35:00,ОНТАЛАП ШАРУА ҚОЖАЛЫҒЫ,ALEM AGRO,"2М-4Х 750, 75% водорастворимый концентрат",20.0,литр,20.03,Заявка оплачена,...,nan,Пестициды,2019,""" ОНТАЛАП "" ШАРУА ҚОЖАЛЫҒЫ-Кызылординская область",Да,True,000000168,430000000,,
9394,4990,Туркестанская область,20190110 00:13:00,ӘБІШЕВ БАЛТАБАЙ ОҢҒАРҰЛЫ,ИП АҚАЛТЫН ЛАБОРАТОРИЯСЫ,Златоглазка (Chrysoperla carnea),35100.0,шт,13.00,Аннулирована,...,nan,Пестициды,2019,ӘБІШЕВ БАЛТАБАЙ ОҢҒАРҰЛЫ-Туркестанская область,Нет,False,nan,610000000,,
47010,26277,Акмолинская область,20190730 14:33:00,ТОО СХП ҒАРЫШ,ТОО КАЗТРАСТКЕМ,"ВАРЯГ, концентрат эмульсии",1420.0,литр,4057.14,Заявка оплачена,...,nan,Пестициды,2019,"Товарищество с ограниченной ответственностью ""...",Нет,False,000000674,110000000,,


In [85]:
subsidies_02.loc[subsidies_02['АА'] != 'Да', ['АА']] = 0
subsidies_02.loc[subsidies_02['АА'] == 'Да', ['АА']] = 1
subsidies_02.loc[subsidies_02['isSubsAaClient'] != 'True', ['isSubsAaClient']] = 0
subsidies_02.loc[subsidies_02['isSubsAaClient'] == 'True', ['isSubsAaClient']] = 1

C:\Users\M_MUKHANOVA\miniconda3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
C:\Users\M_MUKHANOVA\miniconda3\lib\site-packages\pandas\core\ops\array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


In [72]:
print(qqqq.columns)
print(len(qqqq.columns))

qqqq

Index(['index', 'region', 'appDate', 'applicant', 'provider', 'product',
       'volume', 'unit', 'areaOfUsage', 'status', 'typeOfApp',
       'AppAcceptanceDate', 'DateOfRejection', 'subsidies',
       'reasonOfRejection', 'NoteOnNegativeResolution', 'group', 'year',
       'applicantID', 'АА', 'isSubsAaClient', 'codERP', 'cato_Id',
       'applicantIn', 'providerIn'],
      dtype='object')
25


,index,region,appDate,applicant,provider,product,volume,unit,areaOfUsage,status,...,NoteOnNegativeResolution,group,year,applicantID,АА,isSubsAaClient,codERP,cato_Id,applicantIn,providerIn
32945,18497,Кызылординская область,20190813 17:35:00,ОНТАЛАП ШАРУА ҚОЖАЛЫҒЫ,ALEM AGRO,"2М-4Х 750, 75% водорастворимый концентрат",20.0,литр,20.03,Заявка оплачена,...,nan,Пестициды,2019,""" ОНТАЛАП "" ШАРУА ҚОЖАЛЫҒЫ-Кызылординская область",1,0,000000168,430000000,,
9394,4990,Туркестанская область,20190110 00:13:00,ӘБІШЕВ БАЛТАБАЙ ОҢҒАРҰЛЫ,ИП АҚАЛТЫН ЛАБОРАТОРИЯСЫ,Златоглазка (Chrysoperla carnea),35100.0,шт,13.00,Аннулирована,...,nan,Пестициды,2019,ӘБІШЕВ БАЛТАБАЙ ОҢҒАРҰЛЫ-Туркестанская область,0,0,nan,610000000,,
47010,26277,Акмолинская область,20190730 14:33:00,ТОО СХП ҒАРЫШ,ТОО КАЗТРАСТКЕМ,"ВАРЯГ, концентрат эмульсии",1420.0,литр,4057.14,Заявка оплачена,...,nan,Пестициды,2019,"Товарищество с ограниченной ответственностью ""...",0,0,000000674,110000000,,


In [88]:
s_02_sql = ('''SELECT * FROM subsidies_02''')
s_02 = pd.read_sql(s_02_sql, conn)
print(s_02.columns)
s_02

Index(['index', 'region', 'appDate', 'applicant', 'provider', 'applicantIn',
       'providerIn', 'product', 'volume', 'unit', 'areaOfUsage', 'status',
       'typeOfApp', 'AppAcceptanceDate', 'DateOfRejection', 'subsidies',
       'reasonOfRejection', 'NoteOnNegativeResolution', 'group', 'year',
       'applicantID', 'АА', 'isSubsAaClient', 'codERP', 'cato_Id'],
      dtype='object')


,index,region,appDate,applicant,provider,applicantIn,providerIn,product,volume,unit,...,subsidies,reasonOfRejection,NoteOnNegativeResolution,group,year,applicantID,АА,isSubsAaClient,codERP,cato_Id
0,0,г.Шымкент,2019-03-12 15:18:00,КХ BEIIBARYSFRUITS,ТОО КОСАГРОКОММЕРЦ,,,"ВАЛСАГЛИФ, водный раствор",60.000000,литр,...,33530.0,nan,nan,Пестициды,2019,"КХ ""Beiibarysfruits""-Шымкент г.а.",False,False,000000668,790000000
1,1,Павлодарская область,2019-03-12 14:17:00,ТОО ПАВЛОДАРСКАЯ СЕЛЬСКОХОЗЯЙСТВЕННАЯ ОПЫТНАЯ ...,ТОО ЩЕЛКОВО АГРОХИМ-KZ,,,"ПРИМАДОННА, суспензионная эмульсия",350.000000,литр,...,0.0,несоответствие сельхозтоваропроизводителя (сел...,Некорректно заполненная заявка,Пестициды,2019,"Товарищество с ограниченной ответственностью ""...",False,False,000004567,550000000
2,2,Павлодарская область,2019-03-12 13:55:00,ТОО ПАВЛОДАРСКАЯ СЕЛЬСКОХОЗЯЙСТВЕННАЯ ОПЫТНАЯ ...,ТОО ЩЕЛКОВО АГРОХИМ-KZ,,,"ОВСЮГЕН ЭКСТРА, концентрат эмульсии",350.000000,литр,...,0.0,несоответствие сельхозтоваропроизводителя (сел...,Некорректно заполненная заявка,Пестициды,2019,"Товарищество с ограниченной ответственностью ""...",False,False,000001116,550000000
3,3,Павлодарская область,2019-03-12 12:46:00,ТОО ПАВЛОДАРСКАЯ СЕЛЬСКОХОЗЯЙСТВЕННАЯ ОПЫТНАЯ ...,ТОО ЩЕЛКОВО АГРОХИМ-KZ,,,"СПРУТ ЭКСТРА, водный раствор",1250.000000,литр,...,0.0,несоответствие сельхозтоваропроизводителя (сел...,Некорректно заполненная заявка,Пестициды,2019,"Товарищество с ограниченной ответственностью ""...",False,False,000001335,550000000
4,4,Павлодарская область,2019-03-12 12:33:00,ТОО ПАВЛОДАРСКАЯ СЕЛЬСКОХОЗЯЙСТВЕННАЯ ОПЫТНАЯ ...,ТОО ЩЕЛКОВО АГРОХИМ-KZ,,,"ОВСЮГЕН ЭКСТРА, концентрат эмульсии",350.000000,литр,...,0.0,несоответствие сельхозтоваропроизводителя (сел...,Некорректно заполненная заявка,Пестициды,2019,"Товарищество с ограниченной ответственностью ""...",False,False,000001116,550000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160742,174062,Северо-Казахстанская область,2020-06-05 16:22:00,ТОО АСЫЛ ЛОГИСТИКС,ООО АГРОКОМПЛЕКС КУРГАНСЕМЕНА,,,Боевчанка,67280.000000,кг,...,5247839.0,nan,nan,Семена,2020,"ТОО ""АСЫЛ ЛОГИСТИКС""-Северо-Казахстанская область",False,False,000002909,590000000
160743,174063,Костанайская область,2020-06-05 14:37:00,КХ ТРЕТЬЯК НИКОЛАЙ СЕРГЕЕВИЧ,КРЕСТЬЯНСКОЕ ХОЗЯЙСТВО ЛУГОВОЕ,,,ОМСКАЯ 36 ®,20000.199219,кг,...,1560015.0,nan,nan,Семена,2020,"КХ ""Третьяк Николай Сергеевич""-Костанайская об...",False,False,000004383,390000000
160744,174064,Костанайская область,2020-06-05 14:30:00,КРЕСТЬЯНСКОЕ ХОЗЯЙСТВО ТРЕТЬЯК СЕРГЕЙ ЕФИМОВИЧ,КРЕСТЬЯНСКОЕ ХОЗЯЙСТВО ЛУГОВОЕ,,,ОМСКАЯ 36 ®,33000.781250,кг,...,2574060.0,nan,nan,Семена,2020,"Крестьянское хозяйство ""Третьяк Сергей Ефимови...",False,False,000004383,390000000
160745,174065,Северо-Казахстанская область,2020-06-05 14:27:00,ТОО КАНТЕМИР-АГРО,ТОО АБИ-ЖЕР,,,СОЛАР КЛ,2628.000000,кг,...,7358399.0,nan,nan,Семена,2020,"ТОО ""КАНТЕМИР-АГРО""-Северо-Казахстанская область",False,False,000004957,590000000


In [87]:
insertQuery = '''INSERT INTO subsidies_02([index], [region], [appDate], [applicant], [provider], [product], 
                                        [volume], [unit], [areaOfUsage], [status], [typeOfApp], [AppAcceptanceDate], 
                                        [DateOfRejection], [subsidies], [reasonOfRejection], [NoteOnNegativeResolution], [group], [year], 
                                        [applicantID], [АА], [isSubsAaClient], [codERP], [cato_Id],[applicantIn], 
                                        [providerIn]) VALUES (?,?, convert(datetime,?),?,?,?,
                                                            ?,?,?,?,?,?,
                                                            ?,?,?,?,?,?,
                                                            ?,?,?,?,?,?,
                                                            ?);
'''
# try:
#     for t in qqqq.values.tolist():
#         cursor.execute(insertQuery, t)
# except Exception as e:
#     cursor.rollback()
#     print(e)
#     print('transaction rolled back')
# else:
#     print('inserted successfully')
#     cursor.commit()
# finally:
#     if conn.connected == 1:
#         print('conncetion closed')
#         conn.closed()

try:
    conn.autocommit = False
    for t in subsidies_02.values.tolist():
#         print(t)
#         print(len(t))
        cursor.execute(insertQuery, t)
except Exception as e:
    cursor.rollback()
    print(e)
    print('transaction rolled back')
else:
    print('inserted successfully')
    conn.commit()
finally:
    conn.autocommit = True

inserted successfully


In [349]:
# ['г.Шымкент', '20201127 17:45:00', 'КХ ARMAN AGRO', 'ТОО КОСАГРОКОММЕРЦ', 'Минеральное удобрение Kristalon Yellow 13-40-13 (N-13, NH4-8,6, NO3-4,4, P2O5-40, K2O-13, B-0,025, Cu-0,01, Fe-0,07, Mn-0,04, Zn-0,025, Mo-0,004)', 2500.0, 'кг', 3.7, 'Заявка у производителя удобрений', 'Переводная', 'NaT', 'NaT', 1041250.0, 'nan', 'nan', 'Удобрения', '2020', 'КХ ARMAN AGRO-Шымкент г.а.', 'Нет', 'Нет', '000001813', '790000000', '000000000000', '000000000000']
# cursor.execute(insertQuery, ['a', '20201127 17:45:00', 'a', 'a', 'a', 2500, 'кг', 3, 'Заявка у производителя удобрений', 'Переводная', 'NaT', 'NaT', 104, 'nan', 'nan', 'Удобрения', '2020', 'КХ ARMAN AGRO-Шымкент г.а.', 'Нет', 'a', '000001813', '790000000', '000000000000', '000000000000'])
conn.commit()
# qWithKatoCodes.loc[qWithKatoCodes['product'].eq('Минеральное удобрение Kristalon Yellow 13-40-13 (N-13, NH4-8,6, NO3-4,4, P2O5-40, K2O-13, B-0,025, Cu-0,01, Fe-0,07, Mn-0,04, Zn-0,025, Mo-0,004)') & qWithKatoCodes['appDate'].eq('20201127 17:45:00')]

In [347]:
len('Заявка у производителя удобрений')

32

In [49]:
test = pd.melt(qWithKatoCodes.reset_index(), id_vars = ['index']).sort_values(by = ['index'])
test.to_csv(r"\\aafssrv\SHARED-BI\files_aa_dwh_x\testQoldau_02.csv", header = ['index1', 'variable', 'value'], encoding='utf-8', sep='|', quoting = csv.QUOTE_NONE, quotechar="",  escapechar="~")

In [51]:
# testToInsert = pd.read_csv(r"\\aafssrv\SHARED-BI\files_aa_dwh_x\testQoldau_02.csv", encoding="utf-8", sep="|")
test = test.replace('~', '')
test = test.reset_index()
test.columns = ['index0', 'index1', 'variable', 'value']
test.sample(3)

,index0,index1,variable,value
1543702,382371,70168,appDate,17.06.2019 18:03
195218,6252933,8873,codERP,NaN
2395171,3855307,108871,subsidies,"152 250,94"


In [54]:
test=test.applymap(str)

In [303]:
# cursor.execute("DELETE FROM subsidies_02")
# conn.commit()

In [358]:
subsidies_02.loc[subsidies_02['Категория'] == "Удобрения"]

,Область,ДатаПодачиЗаявки,Заявитель,Поставщик,Продукт,Объем,БазоваяЕдиницаИзмерения,ПлощадьИспользованияГа,Статус,ТипЗаявки,ДатаПринятияЗаявки,ДатаОтклонения,СуммаСубсидийТг,ПричинаОтклонения,ПримечаниеКОтрицательномуРешению,Категория,Сезон,applicantID,АА,isSubsAaClient
91039,Туркестанская область,2019-11-30 21:28:00,?БДЫРАЗА? ЕРІК ЕСЕНГЕЛДІ?ЛЫ,ИП МЫРЗАТАЕВ А,"Селитра аммиачная марки Б (N 34,4)",5000.000000,кг,10.500000,Заявка оплачена,Прямая,2019-03-12 11:36:00,NaT,1.495725e+05,nan,nan,Удобрения,2019,?БДЫРАЗА? ЕРІК ЕСЕНГЕЛДІ?ЛЫ-Туркестанская область,Нет,Нет
91040,Алматинская область,2019-11-30 21:23:00,МОЛОТОВ ТУГЛУКЖАН САЙДАЛИМОВИЧ,ТОО NIKINCOM НИК ИНКОМ,"Селитра аммиачная марки Б (N 34,4)",0.400000,тонн,4.000000,Заявка оплачена,Прямая,2019-03-12 12:00:00,NaT,9.800000e+03,nan,nan,Удобрения,2019,МОЛОТОВ ТУГЛУКЖАН САЙДАЛИМОВИЧ-Алматинская обл...,Нет,Нет
91041,Туркестанская область,2019-11-30 20:49:00,ИЗЕНБАЕВ ЖУМАБАЙ СТАНБАЕВИЧ,ТОО КАЙНАР,Суперфосфат простой (P-15),5000.000000,кг,9.370000,Отозвана,Переводная,NaT,NaT,1.187500e+05,nan,nan,Удобрения,2019,ИЗЕНБАЕВ ЖУМАБАЙ СТАНБАЕВИЧ-Туркестанская область,Нет,Нет
91043,Алматинская область,2019-11-30 21:10:00,АБИЛЬДАЕВ САГИДОЛЛА АЛДАБЕРГЕНОВИЧ,ТОО ХИМ. СЕРВИС,Азотно-фосфорно-калийное комплексное минеральн...,1.000000,тонн,4.400000,Заявка оплачена,Прямая,2019-03-12 12:00:00,NaT,5.580357e+04,nan,nan,Удобрения,2019,АБИЛЬДАЕВ САГИДОЛЛА АЛДАБЕРГЕНОВИЧ-Алматинская...,Нет,Нет
91044,Туркестанская область,2019-11-30 20:40:00,ПРОИЗВОДСТВЕННЫЙ КООПЕРАТИВ ШАЯН,ТОО ALTAIR-JS,"Карбамид марки Б (N 46,2)",192.000000,тонн,1662.599976,Заявка оплачена,Прямая,2019-03-12 11:34:00,NaT,9.216000e+06,nan,nan,Удобрения,2019,"Производственный кооператив ""ШАЯН""-Туркестанск...",Нет,Нет
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144132,Алматинская область,2020-10-02 15:07:00,КХ ПОЛУХИН Е.Ю.,ТОО NIKINCOM НИК ИНКОМ,"Селитра аммиачная марки Б (N 34,4)",129.330002,тонн,1096.069946,Заявка оплачена,Прямая,2020-12-02 18:43:00,NaT,4.526655e+06,nan,nan,Удобрения,2020,"КХ ""Полухин Е.Ю.""-Алматинская область",Нет,Нет
144133,Алматинская область,2020-10-02 13:19:00,ТОО АГРО ИМПЭКС АЛАТАУ,ТОО ЮЖНЫЙ ПОЛЕВОД,"Сульфат калия (калий сернокислый) (K2O-51, SO3...",500.000000,кг,2.280000,Заявка оплачена,Прямая,2020-12-02 18:42:00,NaT,6.750000e+04,nan,nan,Удобрения,2020,"Товарищество с ограниченной ответственностью ""...",Нет,Нет
144134,Алматинская область,2020-10-02 12:53:00,ТОО АГРО ИМПЭКС АЛАТАУ,ТОО ЮЖНЫЙ ПОЛЕВОД,"Селитра калиевая техническая марки СХ (N-13,9,...",2500.000000,кг,5.320000,Заявка оплачена,Прямая,2020-12-02 18:42:00,NaT,5.282980e+05,nan,nan,Удобрения,2020,"Товарищество с ограниченной ответственностью ""...",Нет,Нет
144135,Алматинская область,2020-10-02 12:02:00,ТОО АГРО ИМПЭКС АЛАТАУ,ТОО ЮЖНЫЙ ПОЛЕВОД,"Магний сернокислый 7-водный (MgO-16,4, S-12,9)",4300.000000,кг,2.280000,Заявка оплачена,Прямая,2020-12-02 18:41:00,NaT,3.537895e+05,nan,nan,Удобрения,2020,"Товарищество с ограниченной ответственностью ""...",Нет,Нет
